In [11]:
#Import packages
import geopandas as gpd
import pandas as pd
import folium
from folium.plugins import HeatMap,HeatMapWithTime
import sqlite3
import matplotlib.pyplot as plt
%matplotlib inline

In [1]:
#Create pandas dataframe from SQLite data
connect = sqlite3.connect("FPA_FOD_20170508.sqlite")
fires = pd.read_sql_query("SELECT * FROM fires", connect)
fires.head()

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape
0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,0511,Plumas National Forest,...,A,40.036944,-121.005833,5.0,USFS,CA,63,063,Plumas,b'\x00\x01\xad\x10\x00\x00\xe8d\xc2\x92_@^\xc0...
1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.933056,-120.404444,5.0,USFS,CA,61,061,Placer,b'\x00\x01\xad\x10\x00\x00T\xb6\xeej\xe2\x19^\...
2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.984167,-120.735556,13.0,STATE OR PRIVATE,CA,17,017,El Dorado,b'\x00\x01\xad\x10\x00\x00\xd0\xa5\xa0W\x13/^\...
3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.559167,-119.913333,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00\x94\xac\xa3\rt\xfa]...
4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.559167,-119.933056,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00@\xe3\xaa.\xb7\xfb]\...


In [ ]:
#Computationally intense cell

#This cell creates an array wtimedata of form [ [day] , ... , [day] ], where [day] = [[lat,long,size] , ... , [lat,long,size]] 
# ex. wtimedata[x] returns list of all fires on day x; wtimedata[x][y] returns [lat,long,size] of fire y on day x
#     wtimedata[x][y][0] returns latitude of fire y on day x

#Create empty weighted time data array
wtimedata=[]
 
#Append array representing one day of form [day] = [ [lat,long,size],...,[lat,long,size] ]
for i in range(1992,2016):
    for j in range(1,366):
        #Create pd array containing latitudes for fires occurring in year i on day j
        latitude=fires.loc[fires["FIRE_YEAR"]==i].loc[fires["DISCOVERY_DOY"]==j]["LATITUDE"]
        
        #Create pd array containing longitudes for fires occurring in year i on day j
        longitude=fires.loc[fires["FIRE_YEAR"]==i].loc[fires["DISCOVERY_DOY"]==j]["LONGITUDE"]
        
        #Create pd array containing weights for fires occurring in year i on day j
        weight=fires.loc[fires["FIRE_YEAR"]==i].loc[fires["DISCOVERY_DOY"]==j]["FIRE_SIZE"]
        
        #Only append if any fires occurred today
        if (len(latitude)!=0) and (len(longitude)!=0):
            wtimedata.append([list(a) for a in zip(latitude, longitude, weight)])


In [10]:
#Create a map centered/zoomed on the US with stamentoner texture
basemap = folium.Map(location=[50.8283, -98.5795],
                     tiles='stamentoner',
                     zoom_start = 3) 
basemap

In [ ]:
#Create HeatMapWithTime layer from wtimedata
hm = HeatMapWithTime(wtimedata)

#Add layer to basemap
hm.add_to(basemap)

#Version Number/details
v=""

In [ ]:
#Save file; open in firefox for best results
basemap.save("results%s"%v,"heatmapwithtime.html")